In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
X = data.data
Y = data.target

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state=4)
len_x_train = len(X_train)
len_x_test = len(X_test)
len_y_train = len(Y_train)
len_y_test = len(Y_test)

print(f'len_x_train = {len_x_train}')
print(f'len_x_test = {len_x_test}')
print(f'len_y_train = {len_y_train}')
print(f'len_y_trst = {len_y_test}')

len_x_train = 398
len_x_test = 171
len_y_train = 398
len_y_trst = 171


In [5]:
# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.svm import SVC
import numpy as np
import operator
%matplotlib inline

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

# Number of random trials
NUM_TRIALS = 30

# We will use a Support Vector Classifier with "rbf" kernel
svm = SVC()

# Arrays to store scores
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

param_dict = {}
# Loop for each trial
for i in range(NUM_TRIALS):

    # Choose cross-validation techniques for the inner and outer loops,
    # independently of the dataset.
    # E.g "LabelKFold", "LeaveOneOut", "LeaveOneLabelOut", etc.
    inner_cv = StratifiedKFold(n_splits=10, random_state=25, shuffle=True)
    outer_cv = StratifiedKFold(n_splits=10, random_state=25, shuffle=True)

    # Non_nested parameter search and scoring
    clf = GridSearchCV(estimator=svm, param_grid=tuned_parameters, cv=inner_cv, scoring = 'roc_auc', n_jobs=-1)
    clf.fit(X, Y)
    non_nested_scores[i] = clf.best_score_
    print(f'i = {i}, clf.best_params_ = {clf.best_params_}, clf.best_score_ = {clf.best_score_}')
    best_params = str(clf.best_params_)
    
    # Record the parameter count
    if(best_params in param_dict):
        param_dict[best_params] += 1
    else:
        param_dict[best_params] = 1
    
    # Nested CV with parameter optimization
    nested_score = cross_val_score(clf, X=X, y=Y, cv=outer_cv, scoring = 'roc_auc')
    nested_scores[i] = nested_score.mean()

print(f"=============================")
sorted_param_dict = sorted(param_dict.items(), key=operator.itemgetter(1))
for x in sorted_param_dict:
    print(f'{x[0]}, num = {x[1]}')

print(f"=============================")
print(f'mean of nested_score = {nested_score.mean()}')
print(f'mean of non_nested_scores = {non_nested_scores.mean()}')
score_difference = non_nested_scores - nested_scores

print("Average difference of {0:6f} with std. dev. of {1:6f}."
      .format(score_difference.mean(), score_difference.std()))

# Plot scores on each trial for nested and non-nested CV
plt.figure()
plt.subplot(211)
non_nested_scores_line, = plt.plot(non_nested_scores, color='r')
nested_line, = plt.plot(nested_scores, color='b')
plt.ylabel("score", fontsize="14")
plt.legend([non_nested_scores_line, nested_line],
           ["Non-Nested CV", "Nested CV"],
           bbox_to_anchor=(0, .4, .5, 0))
plt.title("Non-Nested and Nested Cross Validation on Iris Dataset",
          x=.5, y=1.1, fontsize="15")

# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend([difference_plot],
           ["Non-Nested CV - Nested CV Score"],
           bbox_to_anchor=(0, 1, .8, 0))
plt.ylabel("score difference", fontsize="14")

plt.show()

i = 0, clf.best_params_ = {'C': 100, 'kernel': 'linear'}, clf.best_score_ = 0.9909918923099942
